# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas`

## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

#### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input)의 embedding vector 간의 **코사인 유사도**를 측정한다.

##### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

### Retrieval 평가
User input에 대해 Vector store에서 검색한 context에 대한 평가

#### Context Precision
- 검색된 문서(context)들 중 **질문과 관련 있는 것들이 얼마나 상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


##### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 질문과의 context간의 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

##### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
    ```bash
    Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
    Precision@2 = 1/2 = 0.5     
    Precision@3 = 2/3 ≈ 0.67    
    Precision@4 = 3/4 = 0.75
    Precision@5 = 3/5 = 0.6
    ```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

#### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

##### 평가방법
1. **정답**에서 주장 문장(claim statement)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장 문장(claim statement)의 정보를 검색된 context들에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)


## RAG Chain 구성
- Vector Store 연결
- Chain 응답 결과
    - 평가를 위해 **Vector Store에서 검색한 context**들과 **LLM 응답**이 출력되도록 chain을 구성한다.

In [ ]:
### Vector Store 연결 1

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma


COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/olympic_info"
DOC_PATH = 'data/olympic.txt'

loader = TextLoader(DOC_PATH, encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", chunk_size=500, chunk_overlap=100
)
docs = loader.load_and_split(splitter)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [41]:
# Vector Store 연결 2

# COLLECTION_NAME = "olympic_info"
# PERSIST_DIRECTORY = "vector_store/olympic_info"

# vector_store = Chroma(
#     embedding_function=embedding_model,
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY
# )

## 평가

In [49]:
user_input = "국제 올림픽 위원회에 대해 설명해주세요."
answer = "국제올림픽위원회(IOC)는 올림픽 활동을 통솔하는 기구로, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 역할을 수행한다. 올림픽 활동은 국제경기연맹(IF), 국가올림픽위원회(NOC), 올림픽조직위원회(OCOG)로 구성된다. 국제경기연맹은 종목별 국제 대회를 관리하는 기구로 FIFA(축구), FIVB(배구)와 같은 단체가 포함되며, 국가올림픽위원회는 각국의 올림픽 활동을 감독하는 기구로 205개가 활동 중이다. 올림픽조직위원회는 대회 준비와 운영을 담당하는 임시 조직으로 대회 종료 후 해산되며, 최종 보고서를 IOC에 제출한다. 올림픽의 공식 언어는 프랑스어, 영어, 개최국의 공용어로, 개막식 등 공식 발표 시 최소 하나의 언어로 진행된다."

# 평가 데이터 셋 만들기

1. 문서를 Loading하고 split하여 Context 들을 만든다.
2. Context들 중 평가에 사용할 것을 Random하게 선택한다.
3. 선택된 context를 기반으로 LLM 모델을 이용해서 질문과 답변을 생성한다. 
4. 생성된 질문과 답변을 검토하여 품질을 높인다. 
   - 질문과 답변 자체가 맞는지 검사한다.
   - 나올만한 질문인 지 확인한다.

- **생성된 질문-답변의 질이 낮으면 좋은 평가를 할 수 없다.**
    - 질문-답변 생성시 사용하는 LLM 모델은 성능이 좋은 것을 사용해야 한다. 
    - 생성된 질문-답변을 사람이 검토해서 품질을 더 높여야 한다.

## 데이터셋에 포함될 내용
- Dataset을 생성할 때는 다음 항목이 들어가야 한다.
    -  user_input: 질문. `string`
    -  reference : 정답. `string`
    -  retrieved_contexts : context. `list(string)`
-  평가할 때 추가할 것
    - response: LLM 모델이 생성한 답변. `string`

### HuggingFace hub에 생성된 평가 Dataset 저장
- 생성한 평가 데이터셋을 HuggingFace hub에 저장하면 datasets을 이용해 load해서 사용 할 수 있다.

### HuggingFace hug의 평가 Datasets load